In [46]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Optimizer

In [17]:
######################################### caution : terminal commands #######################################################

#-------------------------------------empty the datasets forlder before downloading the dataset -------------------------------------

%cd
%cd .keras/datasets/
!rm -r *

/root
/root/.keras/datasets
rm: cannot remove '*': No such file or directory


In [18]:
########################################### download data from given url ###############################################3

dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
data_dir = tf.keras.utils.get_file('nature_12K', origin=dataset_url, extract=True)


3816693760/3816687935 [==============================] - 46s 0us/step


In [19]:
#---------------------------------------------caution : terminal commands ----------------------------------------------

%cd
%cd .keras/datasets/inaturalist_12K
%mv val test
!mkdir valid

/root
/root/.keras/datasets/inaturalist_12K


In [20]:
#################################### split train data into validation set and training set ###################################

data_folder = '/root/.keras/datasets/inaturalist_12K'
os.chdir(data_folder)

folder_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia' ]
for i in range(0,10):
  source = data_folder + "/train/" +folder_names[i]  
  orig_files = os.listdir(source)
  chosen_indexes = random.sample(range(0, len(orig_files)-1), 100)

  destination = data_folder + "/valid/"
  os.chdir(destination)
  os.system('mkdir'+' '+str(folder_names[i]))
  destination = destination + folder_names[i]
  for j in range(0,100):
    shutil.move(   source +"/" + str(   orig_files[   chosen_indexes[j]  ]   )  , destination)


In [21]:
####################################### Correcting the directory location ####################################################

#data_dir = '/root/.keras/datasets/nature_12K'

data_dir = data_dir.split('/')
data_dir.remove('nature_12K')
data_dir.append('inaturalist_12K')
data_dir = '/'.join(data_dir)


In [22]:
#-------------------------------------Taking the train data--------------------------------------------------------
train_data_dir_path = data_dir + '/train'
train_data_dir = pathlib.Path(train_data_dir_path)

#--------------------------------------Taking the validation data-----------------------------------------------
valid_data_dir_path = data_dir + '/valid'
valid_data_dir = pathlib.Path(valid_data_dir_path)

#----------------------------------------Taking the test data-----------------------------------------------
test_data_dir_path = data_dir + '/test'
test_data_dir = pathlib.Path(test_data_dir_path)

In [23]:
#------------------------------------------Parameters for the images-------------------------------------------------
batch_size = 32
img_height = 180
img_width = 180

In [24]:
####################################### converting the image data into dataset ############################################

#-------------------------------------training dataset -----------------------------------------------------
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical"
  )

#----------------------------------------------validation dataset -----------------------------------------------
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  valid_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical"
  )

Found 8999 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


In [51]:
######################################## function for using trained model ##########################################


def get_desired_model(base_model, input_shape=(180,180,3), num_dense_neurons=512,activation="relu",activation2="softmax",dropout=None, pooling="max",l_rate=0.0001): 
    
    base = base_model(input_shape=input_shape,include_top=False,weights='imagenet') 
  
  #--------------------------------------------- perform pooling ---------------------------------------------------
    if pooling == "avg": 
        x = GlobalAveragePooling2D()(base.output) 
    elif pooling == "max": 
        x = GlobalMaxPooling2D()(base.output) 

  #--------------------------------------------- Flatten the output  ---------------------------------------------------

    x = layers.Flatten()(base.output)


  #--------------------------------------------- dropout -------------------------------------------------------
    if dropout is not None: 
        x = Dropout(dropout)(x) 

  #----------------------------------------- add fully connected layer --------------------------------------------------
    x = Dense(num_dense_neurons, activation=activation)(x) 

  #------------------------------------------- and a logistic layer -----------------------------------------------------
    x = Dense(10, activation=activation2)(x) 

  #-------------------------------------------------- get model -----------------------------------------------------
    model = Model(inputs=base.input, outputs=x) 

  #---------------------------------------------- setting the trainable layers --------------------------------------
    for layer in base.layers:
        layer.trainable = False

 #------------------------------------------compiling the model ----------------------------------------------------
    model.compile(loss='categorical_crossentropy', metrics=["accuracy"], optimizer=Adam(l_rate)) 
    
  #--------------------------------------------return model --------------------------------------------------------
    return model

#=========================================== end of get_desired_model function ==================================================

In [52]:
#--------------------------------- using MobileNet model as base model -----------------------------------------------
base_model=MobileNet



#---------------------------------------- calling function get desired model -----------------------------------------
model=get_desired_model(base_model, input_shape=(180,180,3), num_dense_neurons=512 ,activation="relu", activation2="softmax",dropout=None, pooling="max",l_rate=0.0001)




In [53]:
#---------------------------------------- training -------------------------------------------------------------
history = model.fit(train_ds, validation_data = val_ds, epochs = 5)

Epoch 1/5
282/282 [==============================] - 363s 1s/step - loss: 2.6292 - accuracy: 0.2321 - val_loss: 1.9387 - val_accuracy: 0.3230
Epoch 2/5
282/282 [==============================] - 337s 1s/step - loss: 1.6318 - accuracy: 0.4360 - val_loss: 1.9143 - val_accuracy: 0.3300
Epoch 3/5
282/282 [==============================] - 358s 1s/step - loss: 1.3312 - accuracy: 0.5501 - val_loss: 1.9674 - val_accuracy: 0.3400
Epoch 4/5
282/282 [==============================] - 337s 1s/step - loss: 1.0690 - accuracy: 0.6479 - val_loss: 2.0211 - val_accuracy: 0.3360
Epoch 5/5
282/282 [==============================] - 349s 1s/step - loss: 0.8797 - accuracy: 0.7217 - val_loss: 2.2155 - val_accuracy: 0.3310
